In [2]:
"""
HomeGuard Security System Simulator
Author: [Your Name]
Description: A smart home monitoring system that processes sensor readings
             and triggers alerts for security, safety, and comfort issues.
"""

import random
from datetime import datetime
import time

# ========== SYSTEM CONFIGURATION ==========
# Global constants for system modes and alert levels
HOME_MODES = ["HOME", "AWAY", "SLEEP"]  # Possible system modes
ALERT_SEVERITIES = ["LOW", "MEDIUM", "HIGH", "CRITICAL"]  # Alert severity levels

# ========== DATA STRUCTURE FUNCTIONS ==========
def create_sensor(sensor_id, location, sensor_type, threshold=None):
    """
    Creates a sensor dictionary to store sensor information.
    Each sensor has an ID, location, type, and optional threshold value.
    """
    sensor = {
        "id": sensor_id,
        "location": location,
        "type": sensor_type,
        "threshold": threshold,
        "current_value": None  # Stores the latest reading
    }
    return sensor

def create_alert(severity, message, sensor_id, timestamp):
    """
    Creates an alert dictionary to store alert information.
    Alerts have severity, message, sensor ID, and timestamp.
    """
    alert = {
        "severity": severity,
        "message": message,
        "sensor_id": sensor_id,
        "timestamp": timestamp
    }
    return alert

# ========== SENSOR READING FUNCTIONS ==========
def generate_reading(sensor):
    """
    Generates realistic random readings for different sensor types.
    Uses random module to simulate real-world sensor variability.
    """
    sensor_type = sensor["type"] if isinstance(sensor, dict) else sensor.type

    # Temperature sensor: 30-100°F with occasional extremes
    if sensor_type == "temperature":
        if random.random() < 0.1:  # 10% chance of extreme temperature
            return random.choice([random.randint(30, 34), random.randint(96, 100)])
        else:
            return random.randint(50, 85)  # Normal range

    # Motion sensor: Random True/False
    elif sensor_type == "motion":
        return random.choice([True, False])

    # Door sensor: Random OPEN/CLOSED
    elif sensor_type == "door":
        return random.choice(["OPEN", "CLOSED"])

    # Smoke sensor: Mostly CLEAR, occasional DETECTED
    elif sensor_type == "smoke":
        return "DETECTED" if random.random() < 0.05 else "CLEAR"  # 5% chance

    return None

def is_abnormal_reading(sensor, reading_value):
    """
    Checks if a sensor reading is outside normal ranges.
    Uses conditional logic (if/else) to evaluate different sensor types.
    """
    sensor_type = sensor["type"] if isinstance(sensor, dict) else sensor.type

    # Temperature: Abnormal if below 35°F or above 95°F
    if sensor_type == "temperature":
        return reading_value < 35 or reading_value > 95

    # Smoke: Abnormal if smoke is detected
    elif sensor_type == "smoke":
        return reading_value == "DETECTED"

    return False  # Motion and door sensors don't have inherent abnormal states

# ========== SECURITY LOGIC FUNCTIONS ==========
def should_trigger_security_alert(sensor, reading_value, system_mode):
    """
    Determines if a security alert should be triggered.
    Security alerts only occur in AWAY mode for motion/door sensors.
    """
    if system_mode != "AWAY":  # Only check security in AWAY mode
        return False

    sensor_type = sensor["type"] if isinstance(sensor, dict) else sensor.type

    # Motion sensor: Alert if motion detected in AWAY mode
    if sensor_type == "motion":
        return reading_value == True

    # Door sensor: Alert if door opened in AWAY mode
    elif sensor_type == "door":
        return reading_value == "OPEN"

    return False

# ========== ALERT PROCESSING FUNCTION ==========
def process_reading(sensor, reading_value, system_mode):
    """
    Main logic function that processes sensor readings and generates alerts.
    Uses if/else chains to check for different alert conditions.
    Returns a list of alerts (empty list if no alerts needed).
    """
    alerts = []  # List to store generated alerts
    timestamp = datetime.now().strftime("%H:%M:%S")  # Current time for timestamp

    # Extract sensor information (handles both dict and object formats)
    if isinstance(sensor, dict):
        location = sensor["location"]
        sensor_id = sensor["id"]
        sensor_type = sensor["type"]
    else:
        location = sensor.location
        sensor_id = sensor.id
        sensor_type = sensor.type

    # 1. SECURITY ALERTS - Check for break-in risks
    if should_trigger_security_alert(sensor, reading_value, system_mode):
        if sensor_type == "motion":
            alerts.append(create_alert("HIGH",
                f"SECURITY: Motion detected in {location} while in AWAY mode!",
                sensor_id, timestamp))
        elif sensor_type == "door":
            alerts.append(create_alert("HIGH",
                f"SECURITY: {location} opened while in AWAY mode!",
                sensor_id, timestamp))

    # 2. SAFETY ALERTS - Check for hazards
    if sensor_type == "temperature":
        # Frozen pipe risk
        if reading_value < 35:
            alerts.append(create_alert("CRITICAL",
                f"SAFETY: Temperature in {location} is {reading_value}°F - RISK OF FROZEN PIPES!",
                sensor_id, timestamp))
        # Overheating risk
        elif reading_value > 95:
            alerts.append(create_alert("CRITICAL",
                f"SAFETY: Temperature in {location} is {reading_value}°F - RISK OF EQUIPMENT FAILURE!",
                sensor_id, timestamp))

    # Fire risk
    elif sensor_type == "smoke" and reading_value == "DETECTED":
        alerts.append(create_alert("CRITICAL",
            f"SAFETY: Smoke detected in {location} - POSSIBLE FIRE!",
            sensor_id, timestamp))

    # 3. COMFORT NOTIFICATIONS - Only in HOME mode
    if system_mode == "HOME" and sensor_type == "temperature":
        if reading_value < 65 or reading_value > 75:
            # Determine severity based on how far from comfort range
            severity = "MEDIUM" if (60 <= reading_value < 65 or 75 < reading_value <= 80) else "HIGH"
            issue = "too cold" if reading_value < 65 else "too hot"
            alerts.append(create_alert(severity,
                f"COMFORT: Temperature in {location} is {reading_value}°F ({issue})",
                sensor_id, timestamp))

    return alerts  # Return list of alerts (may be empty)

# ========== OUTPUT FUNCTIONS ==========
def trigger_alert(alert):
    """
    Displays alerts with appropriate emojis based on severity.
    Uses dictionary to map severity levels to emoji symbols.
    """
    severity_symbol = {
        "LOW": "ℹ️",
        "MEDIUM": "⚠️",
        "HIGH": "🚨",
        "CRITICAL": "🔥"
    }
    symbol = severity_symbol.get(alert["severity"], "⚠️")  # Default to warning symbol
    print(f"[ALERT!] {symbol} {alert['severity']}: {alert['message']}")

def log_event(message, timestamp=None):
    """
    Logs system events with timestamps.
    Useful for debugging and tracking system activity.
    """
    if timestamp is None:
        timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[LOG] [{timestamp}] {message}")

# ========== SENSOR CLASS ==========
class Sensor:
    """
    Sensor class using Object-Oriented Programming (OOP).
    Encapsulates sensor data and methods into a single object.
    """

    def __init__(self, sensor_id, location, sensor_type, threshold=None):
        """Constructor - initializes sensor properties"""
        self.id = sensor_id
        self.location = location
        self.type = sensor_type
        self.threshold = threshold
        self.current_value = None  # Stores the current reading

    def read(self):
        """Generates and stores a new reading for this sensor"""
        sensor_dict = {"type": self.type, "location": self.location}
        self.current_value = generate_reading(sensor_dict)
        return self.current_value

    def isAbnormal(self):
        """Checks if current reading is abnormal"""
        if self.current_value is None:
            return False
        sensor_dict = {"type": self.type, "threshold": self.threshold}
        return is_abnormal_reading(sensor_dict, self.current_value)

    def reset(self):
        """Resets sensor reading to None"""
        self.current_value = None

    def __str__(self):
        """String representation of sensor (used by print())"""
        status = "No reading" if self.current_value is None else str(self.current_value)
        return f"{self.id} ({self.location}): {status}"

# ========== MAIN SIMULATION FUNCTION ==========
def run_simulation(duration_minutes=5, system_mode="AWAY"):
    """
    Main simulation loop that runs the HomeGuard system.
    Integrates all components: sensors, readings, processing, and alerts.
    """
    print("=" * 50)
    print("=== HomeGuard Security System ===")
    print("=" * 50)
    print(f"Mode: {system_mode}\n")

    # Create sensor objects using the Sensor class
    sensors = [
        Sensor("MOTION_001", "Living Room", "motion"),
        Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
        Sensor("DOOR_001", "Front Door", "door"),
        Sensor("SMOKE_001", "Bedroom", "smoke")
    ]

    # Simulation loop - each iteration = 1 minute
    for minute in range(duration_minutes):
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"\nMinute {minute + 1} - {current_time}")
        print("-" * 30)

        # Process each sensor
        for sensor in sensors:
            reading = sensor.read()  # Get new reading

            # Display reading with formatted output
            if sensor.type == "temperature":
                status = "Normal" if 65 <= reading <= 75 else "Abnormal"
                print(f"[READING] {sensor.location} Temperature: {reading}°F ({status})")
            elif sensor.type == "motion":
                status = "DETECTED" if reading else "No activity"
                print(f"[READING] {sensor.location} Motion: {status}")
            elif sensor.type == "door":
                print(f"[READING] {sensor.location}: {reading}")
            elif sensor.type == "smoke":
                status = "DETECTED" if reading == "DETECTED" else "CLEAR"
                print(f"[READING] {sensor.location} Smoke: {status}")

            # Convert sensor object to dict for processing function
            sensor_dict = {
                "id": sensor.id,
                "location": sensor.location,
                "type": sensor.type,
                "threshold": sensor.threshold
            }

            # Process reading and get alerts
            alerts = process_reading(sensor_dict, reading, system_mode)

            # Trigger all generated alerts
            for alert in alerts:
                trigger_alert(alert)
                if alert["severity"] in ["HIGH", "CRITICAL"]:
                    log_event("Sending notification to homeowner...")

        time.sleep(1)  # Pause for readability

    print("\n" + "=" * 50)
    print("Simulation complete!")
    print("=" * 50)

# ========== MAIN EXECUTION ==========
if __name__ == "__main__":
    """
    Entry point of the program.
    Runs simulations in different modes to demonstrate functionality.
    """
    print("🚀 HomeGuard Security System Simulator")
    print("=" * 50)

    # Test in different modes
    print("\n🔒 Testing AWAY Mode (Security Focus):")
    run_simulation(duration_minutes=3, system_mode="AWAY")

    print("\n🏠 Testing HOME Mode (Comfort Focus):")
    run_simulation(duration_minutes=2, system_mode="HOME")

    print("\n💤 Testing SLEEP Mode (Quiet Mode):")
    run_simulation(duration_minutes=2, system_mode="SLEEP")

    print("\n✅ All tests completed successfully!")

🚀 HomeGuard Security System Simulator

🔒 Testing AWAY Mode (Security Focus):
=== HomeGuard Security System ===
Mode: AWAY


Minute 1 - 15:49:49
------------------------------
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 62°F (Abnormal)
[READING] Front Door: CLOSED
[READING] Bedroom Smoke: CLEAR

Minute 2 - 15:49:50
------------------------------
[READING] Living Room Motion: DETECTED
[ALERT!] 🚨 HIGH: SECURITY: Motion detected in Living Room while in AWAY mode!
[LOG] [15:49:50] Sending notification to homeowner...
[READING] Kitchen Temperature: 67°F (Normal)
[READING] Front Door: CLOSED
[READING] Bedroom Smoke: DETECTED
[ALERT!] 🔥 CRITICAL: SAFETY: Smoke detected in Bedroom - POSSIBLE FIRE!
[LOG] [15:49:50] Sending notification to homeowner...

Minute 3 - 15:49:51
------------------------------
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 53°F (Abnormal)
[READING] Front Door: OPEN
[ALERT!] 🚨 HIGH: SECURITY: Front Door opened while 